# Livestock Analysis

Livestock analysis based on the dataframe previously created using scrape routine and OCR to extract data from auctions pdf's from local auction website.

### Import Modules

In [92]:
# Pandas
import pandas as pd
# Numpy
import numpy as np
# Import math for ceil method in calculating bin
import math
# Plotly
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True) # Offline plotting
# Plotly controls
from IPython.display import display, HTML
# Import scipy and CubicSpline for interpolation
import scipy
from scipy.interpolate import CubicSpline

### Clean Up Dataframe

In [93]:
# Read dataframe
df = pd.read_csv("data.csv")
# Remove useless columns
df.drop(["Unnamed: 0", "age"], axis=1, inplace=True)
# Turn date type from objecto to datetime
df["date"] = df["date"].astype('datetime64[ns]')
# Create year, month and week columns
df["year"] = df["date"].apply(lambda x: x.year)
df["month"] = df["date"].apply(lambda x: x.month)
df["week"] = df["date"].apply(lambda x: x.week)
# Create sex column
df["sex"] = df.category.str.contains("MACHO").replace({True: 1, False: 0})
# Fill NaN values with 0 = Female. Checked manually. The ones missing are in file 17-08-31 and are all female 1st iron.
df["sex"].fillna(0, inplace=True)
# Create iron column
df["iron"] = df.category.str.contains("PRIMERA") + df.category.str.contains("1a")
# Fill NaN values with 0 = Female. Checked manually. The ones missing are in file 17-08-31 and are all female 1st iron.
df["iron"].fillna(1, inplace=True)
df["iron"] = df["iron"].replace({0: 2})
# Change types
df["sex"] = df["sex"].astype(np.int32)
df["iron"] = df["iron"].astype(np.int32)
# Drop category column
df.drop(["category"], axis=1, inplace=True)
# Check if there are no null values in sex column and in iron column
if (df["sex"].isnull().values.any() == False) and (df["iron"].isnull().values.any() == False): print("Ente gut, alles gut!")

Ente gut, alles gut!


### Sales Volumes By Units Sold and COP

Histogram showing the volumes transacted according to the weight. In this specific example we are interested in the males. Segmented by numer of units sold and amount of money transacted in the local currency: COP.

In [94]:
# Dataframe of Males: ML & MC types.
df_1 = df[(df["type"] == "ML") | (df["type"] == "MC")]

In [95]:
# Generate volume histogram by weight
def volume_hist_by_weight(dataframe):
    # Histogram dict
    weight_hist_dict = {}
    # Set bin size for weight
    bin_size = 10
    # Main routine
    for index, row in dataframe.iterrows():
        # Calculate weight bin
        weight_bin = int(bin_size*math.ceil(row["weight"]/bin_size) - (bin_size/2))
        # Update weight_hist_dict
        if weight_bin in weight_hist_dict:
            weight_hist_dict[weight_bin][0] += row["quantity"]
            weight_hist_dict[weight_bin][1] += row["quantity"]*row["price_unit"]
        else:
            weight_hist_dict[weight_bin] = [row["quantity"], row["quantity"]*row["price_unit"]]
    # Sorted keys
    sorted_keys = sorted([k for k in weight_hist_dict])
    # Data dictionary for output
    data_dict = {"unit_volume": [], "cop_volume": [], "bins": []}
    # Populate data dictionary
    for key in sorted_keys:
        data_dict["bins"].append(key)
        data_dict["unit_volume"].append(weight_hist_dict[key][0])
        data_dict["cop_volume"].append(weight_hist_dict[key][1]) 
    # Return data dict
    return data_dict

In [96]:
def plot_volumes(volume_type):
    # Define years
    years = ['2013', '2014', '2015', '2016', '2017']

    # Make figure
    figure = {
        'data': [],
        'layout': {},
        'frames': []
    }

    # Fill in most of layout
    figure['layout']['xaxis'] = {'range': [0, 700], 'title': 'Weight'}
    figure['layout']['yaxis'] = {'title': 'Volume [Units]'}
    figure['layout']['hovermode'] = 'closest'
    figure['layout']['sliders'] = {
        'args': [
            'transition', {
                'duration': 1000,
                'easing': 'cubic-in-out'
            }
        ],
        'initialValue': '2013',
        'plotlycommand': 'animate',
        'values': years,
        'visible': True
    }
    figure['layout']['updatemenus'] = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 5000, 'redraw': True},
                             'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]

    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }

    # Make data
    year = 2013

    # Initialize data
    df_1_year = df_1[df_1['year'] == year]
    data = volume_hist_by_weight(df_1_year)
    data_dict = go.Bar(
        x=data["bins"],
        y=data[volume_type]
    )
    figure['data'].append(data_dict)

    # Make frames
    for year in years:
        frame = {'data': [], 'name': str(year)}

        df_1_year = df_1[df_1['year'] == int(year)]
        data = volume_hist_by_weight(df_1_year)

        data_dict = go.Bar(
            x=data["bins"],
            y=data[volume_type]
        )
        frame['data'].append(data_dict)

        figure['frames'].append(frame)
        slider_step = {'args': [
            [year],
            {'frame': {'duration': 500, 'redraw': True},
             'mode': 'immediate',
           'transition': {'duration': 500}}
         ],
         'label': year,
         'method': 'animate'}
        sliders_dict['steps'].append(slider_step)

    figure['layout']['sliders'] = [sliders_dict]

    iplot(figure)

#### Sales Volume in Units

In [97]:
# Plot unit volume
plot_volumes("unit_volume")

#### Sales Volume in COP

In [98]:
# Plot COP volume
plot_volumes("cop_volume")

### What We Care About: Profit

The point is to find the parameters that give the optimal profit. There are 2 independent variables: the initial weight of the animal and the number of months is kept - this is assuming a fixed number of kilograms per day increase. 

Let:

1. $p_i$ be the purchase price.
2. $p_f$ be the selling price.
3. $\overline{kg}_i$ be the average kilograms per animal on purchase.
4. $\overline{kg}_f$ be the average kilograms per animal when selling.
5. $\Delta t$ be the time lapse in months.
6. $n$ be the number of animals bought.

Then the $ROI$ will be:

$$ROI = \frac{n\cdot (p_f\cdot \overline{kg}_f-p_i\cdot \overline{kg}_i)}{n\cdot p_i\cdot \overline{kg}_i} 
= \frac{p_f\cdot \overline{kg}_f}{p_i\cdot \overline{kg}_i}-1$$

Now, this means that by the end of the operation the capital growth factor will be:

$$CGF = 1 + ROI = \frac{p_f\cdot \overline{kg}_f}{p_i\cdot \overline{kg}_i}$$

Now, given that $\overline{kg}_f$ depends on the purchase weight and the number of months the livestock is fed, we have:

$$ROI = \frac{p_f\cdot (\overline{kg}_i+ \dot{\overline{kg}}\cdot \Delta t)}{p_i\cdot \overline{kg}_i}-1 = \frac{p_f}{p_i}\cdot \left(\frac{\dot{\overline{kg}}\cdot \Delta t}{\overline{kg}_i} +1\right)-1$$

The former equation shows the $ROI$ of the operation, where: $\dot{\overline{kg}}$ is the average weight gain per month per unit. However, this equation depends on the time lapse - number of months - of the operation. In order to have a standard measure lets look at the annualized $ROI$.

$$CGF_{ANNUALIZED} = \left[\frac{p_f}{p_i}\cdot \left(\frac{\dot{\overline{kg}}\cdot \Delta t}{\overline{kg}_i} +1\right)\right]^{\left(\frac{12}{\Delta t}\right)}$$

In [99]:
# Make a dataframe with index of date in the format YYYY-MM and the columns the different weight bins.
# The elements of the matrix will correspond to the price of the livestock unit for that specific time period.
def get_price_dataframe(df, livestock_types):
    # Filter condition
    filter_condition = False
    for livestock_type in livestock_types:
        filter_condition = filter_condition | (df["type"] == livestock_type)
    # Filtered dataframe
    df = df[filter_condition]
    # Select only relevant columns
    df = df[[ "year", "month", "week", "weight", "quantity", "price_avg"]]
    # Make price_avg_numerator 
    df["price_avg_numerator"] = df["quantity"]*df["price_avg"] 
    # Make bin
    bin_size = 10
    df["bin"] = df["weight"].apply(lambda weight: int(bin_size*math.ceil(weight/bin_size) - (bin_size/2)))
    # Grouped by year, month and bin
    df_grouped = df.groupby(["year", "month", "bin"])
    # Prices by year, month and weight bin
    df_prices_by_year_month_bin = pd.DataFrame([])
    df_prices_by_year_month_bin["price"] = (df_grouped["price_avg_numerator"].sum()/df_grouped["quantity"].sum()).apply(lambda x: round(x))
    # Output dataframe
    return df_prices_by_year_month_bin

In [100]:
# Get df prices
df_prices = get_price_dataframe(df, ["ML", "MC"])
# Define weight support
weight_support = {"min": 150, "max": 400, "bin_size": 10}

In [101]:
# High price filter
high_price_filter = (df_prices["price"] > 6000).values

In [102]:
# Drop rows where prices are too high. This are errors in pdf's. Human errors.
for index, row in df_prices[high_price_filter].iterrows():
    df_prices.drop((index), inplace=True)

In [103]:
# Make a copy of df_prices original
df_prices_original = pd.DataFrame.copy(df_prices)

In [104]:
# Find greates_min and least_max for weight variable
greatest_min = None
least_max = None
weight_bin_size = 10
# Get years
years = df_prices.index.levels[0]
# Loop through each year
for year in years:
    # Select months of the year
    months = sorted(list(set(df_prices.loc[(year)].index.get_level_values(0))))
    # Loop through each month available for the year
    for month in months:
        # Get x (weight bins) and y (prices) variables for the corresponding (year, month) tuple 
        x = list(df_prices.loc[(year,month)]["price"].index)
        y = list(df_prices.loc[(year,month)]["price"].values)
        # If greatest_min and least_max are not initialized. Initialize them.
        if greatest_min == None: greatest_min = min(x)
        if least_max == None: least_max = max(x)
        # If greatest_min and least_max are initialized. Update them.
        if min(x) > greatest_min: greatest_min = min(x)
        if max(x) < least_max: least_max = max(x)
# On done print greatest_min and least_max
print("Greatest minimum:", greatest_min)
print("Least maximum:", least_max)
# Create ary from greatest_min to least_max in bin_size steps
weight_support_bins = range(greatest_min, least_max + weight_bin_size, weight_bin_size)

Greatest minimum: 115
Least maximum: 395


In [105]:
# Fill in the gaps for the weights in weight_support_bins
for year in years:
    # Select months of the year
    months = sorted(list(set(df_prices.loc[(year)].index.get_level_values(0))))
    # Loop through each month available for the year
    for month in months:
        # Get x (weight bins) and y (prices) variables for the corresponding (year, month) tuple 
        x = list(df_prices.loc[(year,month)]["price"].index)
        y = list(df_prices.loc[(year,month)]["price"].values)
        missing_bins = sorted(list(set(weight_support_bins)-set(x)))
        # Create spline interpolation and calculate for the weight_support_bins
        cs = CubicSpline(x, y)
        # Fill in the gaps
        for w in missing_bins:
            df_prices.loc[(year,month,w)] = int(cs(w))
# Sort df_prices dataframe
df_prices.sort_index(inplace=True)

/Users/pedroatencio/anaconda/envs/py361/lib/python3.6/site-packages/ipykernel_launcher.py:8: PerformanceWarning:

indexing past lexsort depth may impact performance.

/Users/pedroatencio/anaconda/envs/py361/lib/python3.6/site-packages/ipykernel_launcher.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [106]:
# Get df_prices_support i.e. those for which weight is in the range 150 - 300 kg.
df_prices_support = df_prices[(df_prices.index.get_level_values(2) >= min(weight_support_bins)) & (df_prices.index.get_level_values(2) <= max(weight_support_bins))]

In [107]:
# Drop the month of September 2017 because data is not complete
df_prices_support.drop((2017,9), inplace=True)

/Users/pedroatencio/anaconda/envs/py361/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [108]:
# Define dataframe
df_roi_annualized = pd.DataFrame.copy(df_prices_support)
# Set variables for ROI calculations
weight_gain_per_month = 20 # Set to 20 kg per month. It will be a multiple of 10 and will jump 2 bins/month 
len_df_roi_annualized = len(df_roi_annualized)
# Add columns for the twelve months. Initialize them with value: NaN
for i in range(12): df_roi_annualized["month_{0}".format(i+1)] = math.nan

# Indexes of dataframe
indexes_df_roi_annualized = df_roi_annualized.index

# Loop through each row of the df_roi_annualized dataframe to populate it
for i in range(len_df_roi_annualized):
    # Get i-th row
    row_i = df_roi_annualized.iloc[i]
    # Get year, month and initial weight_bin
    year_i, month_i, weight_bin_i = row_i.name
    # Get the initial price
    price_i = row_i["price"] 
    # Calculate each ROI_annualized for the months duration for that row 
    for j in range(12):
        # Months duration
        delta_t_months = j+1
        # Final weight bin obtained in the months duration
        weight_bin_f = weight_bin_i + weight_gain_per_month * delta_t_months
        # Final date after the months duration
        year_f = year_i
        month_f = month_i + delta_t_months
        if month_f > 12: # If the month goes overboard to the next year => Recalculate year and month
            year_f = year_i + 1
            month_f = month_f - 12
        # Check that the corresponding final index exists in the dataframe, i.e. that
        # the price exists for the final values of year, month and weight
        if (year_f, month_f, weight_bin_f) in indexes_df_roi_annualized:
            # Get final price
            price_f = df_roi_annualized.loc[(year_f, month_f, weight_bin_f), "price"]
            # Calculate roi_annualized for the i-th row and column: delta_t_months
            roi_annualized_i_j = (((price_f*weight_bin_f)/(price_i*weight_bin_i)-1)*0.4+1)**(12/delta_t_months)
            # Set roi_annualized for the i-th row and column: delta_t_months into df_roi_annualized dataframe if is more than 20%
            df_roi_annualized.loc[(year_i,month_i,weight_bin_i), "month_{0}".format(delta_t_months)] = roi_annualized_i_j

In [110]:
# Select those with high price
df_roi_annualized.loc[(2017,2)]

,price,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
bin,,,,,,,,,,,,,
115,4267,4.063133,2.880504,3.173066,2.913878,2.264871,2.269444,NaN,NaN,NaN,NaN,NaN,NaN
125,4434,4.737547,3.232968,2.209796,2.404695,1.896848,2.220366,NaN,NaN,NaN,NaN,NaN,NaN
135,4185,4.920461,2.731649,2.945345,2.356098,2.236428,2.044205,NaN,NaN,NaN,NaN,NaN,NaN
145,5350,1.088643,1.505504,1.603799,1.575983,1.555803,1.504311,NaN,NaN,NaN,NaN,NaN,NaN
155,4100,6.924161,3.779251,2.317804,2.489214,2.097542,2.046318,NaN,NaN,NaN,NaN,NaN,NaN
165,4655,2.855013,1.876844,1.755580,1.782301,1.688126,1.629862,NaN,NaN,NaN,NaN,NaN,NaN
175,5092,1.137433,1.144906,1.614528,1.531367,1.395119,1.356390,NaN,NaN,NaN,NaN,NaN,NaN
185,4400,2.821579,1.828702,2.123412,1.857368,1.536730,1.568544,NaN,NaN,NaN,NaN,NaN,NaN
195,4593,2.836434,2.143186,1.763319,1.653088,1.450006,1.496618,NaN,NaN,NaN,NaN,NaN,NaN
